In [ ]:
!unzip "/content/drive/MyDrive/hacker_earth/lupin_hiring_c/8b9aafccf7da11ee.zip"

Archive:  /content/drive/MyDrive/hacker_earth/lupin_hiring_c/8b9aafccf7da11ee.zip
   creating: dataset/
  inflating: dataset/sample_submission.csv  
  inflating: dataset/train.csv       
  inflating: dataset/test.csv        


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
train = pd.read_csv('/content/dataset/train.csv')
test = pd.read_csv('/content/dataset/test.csv')
ss = pd.read_csv('/content/dataset/sample_submission.csv')

In [ ]:

from sklearn.preprocessing import LabelEncoder

# Function to label encode object columns, excluding a specific column
def label_encode_excluding(df, exclude_column):
    # Select columns with object dtype, excluding the specified column
    object_cols = df.select_dtypes(include=['object']).columns
    object_cols = [col for col in object_cols if col != exclude_column]

    # Create a label encoder
    le = LabelEncoder()

    # Label encode the selected object columns
    for col in object_cols:
        df[col] = le.fit_transform(df[col])

    return df

# Label encode object columns, excluding the 'id' column



# Convert object columns to numeric
train_new = label_encode_excluding(train,'candidate_id')
test_new  = label_encode_excluding(test,'candidate_id')

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from math import sqrt
from lightgbm import LGBMRegressor
import xgboost as xgb
import datetime
from sklearn.preprocessing import LabelEncoder

In [ ]:
train_new['kfold'] = -1

In [ ]:
from sklearn import model_selection
kf=model_selection.KFold(n_splits=8,shuffle=True,random_state=42)
for fold ,(train_indicies,valid_indicies) in enumerate(kf.split(X=train_new)):
  train_new.loc[valid_indicies,'kfold']=fold

In [ ]:
ff=['candidate_id','kfold','triglyceride_lvl']

In [ ]:

ff=['candidate_id','kfold','triglyceride_lvl']
useful_feature=[c for c in train_new.columns if c not in ff]
useful_feature

['gender',
 'age',
 'height_in_cm',
 'weight_in_lbs',
 'left_eyesight_lvl',
 'right_eyesight_lvl',
 'can_hear_left_ear',
 'can_hear_right_ear',
 'blood_pressure_lvl1',
 'blood_pressure_lvl2',
 'glucose_lvl',
 'total_cholestrol',
 'good_cholestrol_lvl',
 'bad_cholestrol_lvl',
 'hemoglobin_lvl',
 'urea_lvl',
 'creatinine_lvl',
 'liver_enzyme_lvl1',
 'liver_enzyme_lvl2',
 'smoking_habit',
 'drinking_habit',
 'residential_area']

In [ ]:
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb

In [ ]:
params = {'learning_rate': 0.07105259171384938, 'max_depth': 10, 'min_child_weight': 9, 'subsample': 0.806661217402246, 'colsample_bytree': 0.9598863844008446,
                'gamma': 5.114259067634118e-08, 'lambda': 0.0021798733374827485, 'alpha': 1.7098143106224159e-07}

In [ ]:
maes = []
all_test_predictions = []
test_p = []
final_test_pr=[]
final_valid_pr={}
# Train and evaluate the model using KFold
# Train and evaluate the model using KFold
for fold in range(8):
    xtrain=train_new[train_new.kfold!=fold].reset_index(drop=True)
    xvalid=train_new[train_new.kfold==fold].reset_index(drop=True)

    valid_ids=xvalid.candidate_id.values.tolist()

    ytrain=xtrain['triglyceride_lvl']
    yvalid=xvalid['triglyceride_lvl']

    xtrain=xtrain[useful_feature]
    xvalid=xvalid[useful_feature]

    # Initialize and train the XGBoost model
    model = xgb.XGBRegressor(objective='reg:squarederror', **params,n_estimators=2000, random_state=42)
    model.fit(xtrain, ytrain,
              eval_set=[(xvalid, yvalid)],
              early_stopping_rounds=500,
              verbose=20)

    # Predict on the test set
    y_pred = model.predict(xvalid)
    final_valid_pr.update(dict(zip(valid_ids,y_pred)))
    # Calculate MAE
    mae = mean_absolute_error(yvalid, y_pred)
    maes.append(mae)

    # Store the predictions
    #all_test_predictions.extend(y_pred)



    test_pred = model.predict(test_new[useful_feature])

    test_p.append(test_pred)

    print(f'Fold MAE: {mae}')

# Calculate mean and std of MAEs
mean_mae = np.mean(maes)
std_mae = np.std(maes)

print(f'Final MAE Mean: {mean_mae}')
print(f'Final MAE Std: {std_mae}')

# Calculate the mean of all test predictions
mean_test_prediction = np.mean(test_p, axis=0)
print(f'Mean of all test predictions: {mean_test_prediction}')

/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[0]	validation_0-rmse:76.91519
[20]	validation_0-rmse:35.24644
[40]	validation_0-rmse:28.17810
[60]	validation_0-rmse:26.88665
[80]	validation_0-rmse:26.62533
[100]	validation_0-rmse:26.55148
[120]	validation_0-rmse:26.46734
[140]	validation_0-rmse:26.42228
[160]	validation_0-rmse:26.39016
[180]	validation_0-rmse:26.38271
[200]	validation_0-rmse:26.38934
[220]	validation_0-rmse:26.38288
[240]	validation_0-rmse:26.39602
[260]	validation_0-rmse:26.39815
[280]	validation_0-rmse:26.41997
[300]	validation_0-rmse:26.43905
[320]	validation_0-rmse:26.44131
[340]	validation_0-rmse:26.48734
[360]	validation_0-rmse:26.50779
[380]	validation_0-rmse:26.55449
[400]	validation_0-rmse:26.56884
[420]	validation_0-rmse:26.56118
[440]	validation_0-rmse:26.57456
[460]	validation_0-rmse:26.58514
[480]	validation_0-rmse:26.56655
[500]	validation_0-rmse:26.55459
[520]	validation_0-rmse:26.55949
[540]	validation_0-rmse:26.55963
[560]	validation_0-rmse:26.55305
[580]	validation_0-rmse:26.56488
[600]	validation

/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[20]	validation_0-rmse:35.44347
[40]	validation_0-rmse:27.81366
[60]	validation_0-rmse:26.55870
[80]	validation_0-rmse:26.12731
[100]	validation_0-rmse:25.92519
[120]	validation_0-rmse:25.82665
[140]	validation_0-rmse:25.79658
[160]	validation_0-rmse:25.78094
[180]	validation_0-rmse:25.79160
[200]	validation_0-rmse:25.82520
[220]	validation_0-rmse:25.83586
[240]	validation_0-rmse:25.84830
[260]	validation_0-rmse:25.88145
[280]	validation_0-rmse:25.86429
[300]	validation_0-rmse:25.89587
[320]	validation_0-rmse:25.90381
[340]	validation_0-rmse:25.92154
[360]	validation_0-rmse:25.92385
[380]	validation_0-rmse:25.95674
[400]	validation_0-rmse:25.98641
[420]	validation_0-rmse:25.97799
[440]	validation_0-rmse:25.98934
[460]	validation_0-rmse:26.00611
[480]	validation_0-rmse:26.04249
[500]	validation_0-rmse:26.06325
[520]	validation_0-rmse:26.07204
[540]	validation_0-rmse:26.06586
[560]	validation_0-rmse:26.08342
[580]	validation_0-rmse:26.09457
[600]	validation_0-rmse:26.10851
[620]	validati

/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


...

[20]	validation_0-rmse:39.37622
[40]	validation_0-rmse:32.06302
[60]	validation_0-rmse:30.63460
[80]	validation_0-rmse:30.16312
[100]	validation_0-rmse:29.93513
[120]	validation_0-rmse:29.78796
[140]	validation_0-rmse:29.69930
[160]	validation_0-rmse:29.68740
[180]	validation_0-rmse:29.69874
[200]	validation_0-rmse:29.68759
[220]	validation_0-rmse:29.68235
[240]	validation_0-rmse:29.64027
[260]	validation_0-rmse:29.66836
[280]	validation_0-rmse:29.66226
[300]	validation_0-rmse:29.64944
[320]	validation_0-rmse:29.63070
[340]	validation_0-rmse:29.63536
[360]	validation_0-rmse:29.62189
[380]	validation_0-rmse:29.62531
[400]	validation_0-rmse:29.63277
[420]	validation_0-rmse:29.65378
[440]	validation_0-rmse:29.65577
[460]	validation_0-rmse:29.62727
[480]	validation_0-rmse:29.61846
[500]	validation_0-rmse:29.61487
[520]	validation_0-rmse:29.62702
[540]	validation_0-rmse:29.61961
[560]	validation_0-rmse:29.60861
[580]	validation_0-rmse:29.60196
[600]	validation_0-rmse:29.58329
[620]	validati

/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[20]	validation_0-rmse:36.90935
[40]	validation_0-rmse:30.44336
[60]	validation_0-rmse:29.26210
[80]	validation_0-rmse:28.98088
[100]	validation_0-rmse:28.84444
[120]	validation_0-rmse:28.76310
[140]	validation_0-rmse:28.74231
[160]	validation_0-rmse:28.72556
[180]	validation_0-rmse:28.74398
[200]	validation_0-rmse:28.77546
[220]	validation_0-rmse:28.76409
[240]	validation_0-rmse:28.80125
[260]	validation_0-rmse:28.78893
[280]	validation_0-rmse:28.81853
[300]	validation_0-rmse:28.82565
[320]	validation_0-rmse:28.82075
[340]	validation_0-rmse:28.81162
[360]	validation_0-rmse:28.83858
[380]	validation_0-rmse:28.82511
[400]	validation_0-rmse:28.83717
[420]	validation_0-rmse:28.84281
[440]	validation_0-rmse:28.85590
[460]	validation_0-rmse:28.87290
[480]	validation_0-rmse:28.88947
[500]	validation_0-rmse:28.88367
[520]	validation_0-rmse:28.88505
[540]	validation_0-rmse:28.88355
[560]	validation_0-rmse:28.87710
[580]	validation_0-rmse:28.85979
[600]	validation_0-rmse:28.87169
[620]	validati

/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[20]	validation_0-rmse:36.05193
[40]	validation_0-rmse:28.46952
[60]	validation_0-rmse:27.10913
[80]	validation_0-rmse:26.60132
[100]	validation_0-rmse:26.43608
[120]	validation_0-rmse:26.32706
[140]	validation_0-rmse:26.22952
[160]	validation_0-rmse:26.14889
[180]	validation_0-rmse:26.12995
[200]	validation_0-rmse:26.13355
[220]	validation_0-rmse:26.10617
[240]	validation_0-rmse:26.08978
[260]	validation_0-rmse:26.05996
[280]	validation_0-rmse:26.04576
[300]	validation_0-rmse:26.04229
[320]	validation_0-rmse:26.05378
[340]	validation_0-rmse:26.04961
[360]	validation_0-rmse:26.06068
[380]	validation_0-rmse:26.04529
[400]	validation_0-rmse:26.05224
[420]	validation_0-rmse:26.06390
[440]	validation_0-rmse:26.08086
[460]	validation_0-rmse:26.06236
[480]	validation_0-rmse:26.07023
[500]	validation_0-rmse:26.06134
[520]	validation_0-rmse:26.06324
[540]	validation_0-rmse:26.06014
[560]	validation_0-rmse:26.06877
[580]	validation_0-rmse:26.07391
[600]	validation_0-rmse:26.07027
[620]	validati

In [ ]:
ss = pd.DataFrame()
ss['candidate_id'] = test_new.candidate_id
ss['triglyceride_lvl_xgb'] = mean_test_prediction

In [ ]:
final_valid_pred=pd.DataFrame.from_dict(final_valid_pr ,orient="index").reset_index()
final_valid_pred.columns=['candidate_id','triglyceride_lvl_xgb_tuned']

In [ ]:
final_valid_pred.to_csv('xgb_tuned_tr.csv',index=False)

In [ ]:
import lightgbm as lgb

In [ ]:
len(final_valid_pr)

22400

In [ ]:
# Lists to store MAEs and predictions
maes = []
test_p = []
final_valid_pr={}

for fold in range(8):
    xtrain=train_new[train_new.kfold!=fold].reset_index(drop=True)
    xvalid=train_new[train_new.kfold==fold].reset_index(drop=True)

    valid_ids=xvalid.candidate_id.values.tolist()

    ytrain=xtrain['triglyceride_lvl']
    yvalid=xvalid['triglyceride_lvl']

    xtrain=xtrain[useful_feature]
    xvalid=xvalid[useful_feature]

    # Create LightGBM datasets
    train_data = lgb.Dataset(xtrain, label=ytrain)
    valid_data = lgb.Dataset(xvalid, label=yvalid, reference=train_data)

    # Define parameters
    params = {
        'objective': 'regression',
        'metric': 'mae',
        'boosting_type': 'gbdt',
        'random_state': 42,
        'verbose': -1  # Suppress LightGBM warnings
    }

    # Train the model with early stopping
    model = lgb.train(params, train_data,
                      valid_sets=[valid_data],
                      num_boost_round=2000,
                      callbacks=[lgb.early_stopping(stopping_rounds=500), lgb.log_evaluation(20)])

    # Predict on the test set
    y_pred = model.predict(xvalid, num_iteration=model.best_iteration)

    final_valid_pr.update(dict(zip(valid_ids,y_pred)))
    # Calculate MAE
    mae = mean_absolute_error(yvalid, y_pred)
    maes.append(mae)

    # Predict on the new test set for each fold
    test_pred = model.predict(test_new[useful_feature], num_iteration=model.best_iteration)

    # Store the predictions
    test_p.append(test_pred)

    print(f'Fold MAE: {mae}')

# Calculate mean and std of MAEs
mean_mae = np.mean(maes)
std_mae = np.std(maes)

print(f'Final MAE Mean: {mean_mae}')
print(f'Final MAE Std: {std_mae}')

# Aggregate the predictions to get the mean value for each test sample
mean_test_prediction = np.mean(test_p, axis=0)
print(f'Mean of all test predictions: {mean_test_prediction}')

Training until validation scores don't improve for 500 rounds
[20]	valid_0's l1: 29.5371
[40]	valid_0's l1: 21.4102
[60]	valid_0's l1: 18.2889
[80]	valid_0's l1: 17.129
[100]	valid_0's l1: 16.4341
[120]	valid_0's l1: 15.9968
[140]	valid_0's l1: 15.6403
[160]	valid_0's l1: 15.3863
[180]	valid_0's l1: 15.1515
[200]	valid_0's l1: 14.985
[220]	valid_0's l1: 14.8478
[240]	valid_0's l1: 14.7461
[260]	valid_0's l1: 14.6488
[280]	valid_0's l1: 14.581
[300]	valid_0's l1: 14.5001
[320]	valid_0's l1: 14.4347
[340]	valid_0's l1: 14.3849
[360]	valid_0's l1: 14.3365
[380]	valid_0's l1: 14.2642
[400]	valid_0's l1: 14.2471
[420]	valid_0's l1: 14.1927
[440]	valid_0's l1: 14.1494
[460]	valid_0's l1: 14.1421
[480]	valid_0's l1: 14.1398
[500]	valid_0's l1: 14.1162
[520]	valid_0's l1: 14.0789
[540]	valid_0's l1: 14.0934
[560]	valid_0's l1: 14.0821
[580]	valid_0's l1: 14.0286
[600]	valid_0's l1: 14.0323
[620]	valid_0's l1: 14.0075
[640]	valid_0's l1: 14.0032
[660]	valid_0's l1: 13.9868
[680]	valid_0's l1: 1

In [ ]:
ss

,candidate_id,triglyceride_lvl_xgb
0,CAN_22401,122.495407
1,CAN_22402,73.831909
2,CAN_22403,63.453911
3,CAN_22404,216.933228
4,CAN_22405,172.554306
...,...,...
9595,CAN_31996,37.773037
9596,CAN_31997,144.394638
9597,CAN_31998,78.454346
9598,CAN_31999,78.415993


In [ ]:
ss['triglyceride_lvl_lgbm'] = mean_test_prediction

In [ ]:
final_valid_pred

,candidate_id,triglyceride_lvl_xgb_tuned
0,CAN_18,48.777412
1,CAN_35,95.671501
2,CAN_36,91.923599
3,CAN_45,43.925785
4,CAN_55,69.359116
...,...,...
22395,CAN_22384,72.576134
22396,CAN_22390,51.681309
22397,CAN_22397,213.172836
22398,CAN_22398,255.177628


In [ ]:
type(final_valid_pr)

dict

In [ ]:
final_valid_pr

{'CAN_18': 54.55576216229793,
 'CAN_35': 97.55354683941688,
 'CAN_36': 94.08242484876448,
 'CAN_45': 43.4772359377537,
 'CAN_55': 75.14022438838731,
 'CAN_60': 71.71951105796637,
 'CAN_102': 136.65740296527352,
 'CAN_120': 263.44298052722587,
 'CAN_128': 48.25517868699869,
 'CAN_134': 100.0981595490075,
 'CAN_135': 158.16563899185567,
 'CAN_140': 161.20773267365402,
 'CAN_181': 78.04283413276595,
 'CAN_195': 82.13894962189883,
 'CAN_200': 189.0112922527887,
 'CAN_201': 259.22808226161067,
 'CAN_209': 193.36922885188773,
 'CAN_211': 101.08971453416545,
 'CAN_218': 73.83508035411961,
 'CAN_219': 126.27726816422854,
 'CAN_222': 65.01607004170067,
 'CAN_231': 53.57813341221477,
 'CAN_248': 90.9561563293183,
 'CAN_252': 68.97921082599828,
 'CAN_260': 338.5742772773272,
 'CAN_264': 106.16956165614663,
 'CAN_266': 322.17729690131216,
 'CAN_275': 97.67535266715898,
 'CAN_279': 209.63843443733458,
 'CAN_284': 326.45146499856446,
 'CAN_291': 121.65677221822614,
 'CAN_292': 70.26706814771069,
 'C

In [ ]:
my_dict=final_valid_pr.copy()

In [ ]:
# Check if the dictionary is not empty
if my_dict:
    # Get the last key added
    last_key = list(my_dict.keys())[-1]
    # Get the value associated with the last key
    last_value = my_dict[last_key]
    print(last_value)
else:
    print("The dictionary is empty.")


136.3834917883417


In [ ]:
last_key = final_valid_pr[-1]
last_value = my_dict[last_key]

In [ ]:
final_valid_pred['triglyceride_lvl_lgbm'] = values

In [ ]:
final_valid_pred

,candidate_id,triglyceride_lvl_xgb_tuned,triglyceride_lvl_lgbm
0,CAN_18,48.777412,54.555762
1,CAN_35,95.671501,97.553547
2,CAN_36,91.923599,94.082425
3,CAN_45,43.925785,43.477236
4,CAN_55,69.359116,75.140224
...,...,...,...
22395,CAN_22384,72.576134,75.685533
22396,CAN_22390,51.681309,40.969546
22397,CAN_22397,213.172836,222.469224
22398,CAN_22398,255.177628,282.779198


In [ ]:
values = list(final_valid_pr.values())
values

[54.55576216229793,
 97.55354683941688,
 94.08242484876448,
 43.4772359377537,
 75.14022438838731,
 71.71951105796637,
 136.65740296527352,
 263.44298052722587,
 48.25517868699869,
 100.0981595490075,
 158.16563899185567,
 161.20773267365402,
 78.04283413276595,
 82.13894962189883,
 189.0112922527887,
 259.22808226161067,
 193.36922885188773,
 101.08971453416545,
 73.83508035411961,
 126.27726816422854,
 65.01607004170067,
 53.57813341221477,
 90.9561563293183,
 68.97921082599828,
 338.5742772773272,
 106.16956165614663,
 322.17729690131216,
 97.67535266715898,
 209.63843443733458,
 326.45146499856446,
 121.65677221822614,
 70.26706814771069,
 106.98314424664606,
 40.77969103759019,
 182.36837321499934,
 98.85853855708898,
 176.83216395837343,
 76.74672987265362,
 92.1978140612263,
 118.68029663829763,
 139.16290499130199,
 65.39999078316384,
 81.78296212929226,
 236.94861973935917,
 320.02501633263955,
 93.34767821345058,
 64.88548449913299,
 106.18129130353512,
 63.27813902870046,
 9

<function dict.values>

In [ ]:
final_valid_pred=pd.DataFrame.from_dict(final_valid_pr ,orient="index").reset_index()
final_valid_pred.columns=['candidate_id','triglyceride_lvl_lgbm']

In [ ]:
final_valid_pred.to_csv('lgbm_1.csv',index=False)

#catboost

In [ ]:
!pip install catboost

In [ ]:
from catboost import CatBoostRegressor

In [ ]:
maes = []
test_p = []
final_valid_pr ={}

# Train and evaluate the model using KFold
for fold in range(8):
    xtrain=train_new[train_new.kfold!=fold].reset_index(drop=True)
    xvalid=train_new[train_new.kfold==fold].reset_index(drop=True)

    valid_ids=xvalid.candidate_id.values.tolist()

    ytrain=xtrain['triglyceride_lvl']
    yvalid=xvalid['triglyceride_lvl']

    xtrain=xtrain[useful_feature]
    xvalid=xvalid[useful_feature]

    # Initialize and train the CatBoost model
    model = CatBoostRegressor(
        iterations=2000,
        learning_rate=0.03,
        depth=6,
        eval_metric='MAE',
        random_seed=42,
        early_stopping_rounds=500,
        verbose=100
    )
    model.fit(xtrain, ytrain, eval_set=(xvalid, yvalid), use_best_model=True)

    y_pred = model.predict(xvalid)
    final_valid_pr.update(dict(zip(valid_ids,y_pred)))
    # Calculate MAE
    mae = mean_absolute_error(yvalid,y_pred)
    maes.append(mae)

    # Predict on the new test set for each fold
    test_pred = model.predict(test_new[useful_feature])

    # Store the predictions
    test_p.append(test_pred)

    print(f'Fold MAE: {mae}')

# Calculate mean and std of MAEs
mean_mae = np.mean(maes)
std_mae = np.std(maes)

print(f'Final MAE Mean: {mean_mae}')
print(f'Final MAE Std: {std_mae}')

# Aggregate the predictions to get the mean value for each test sample
mean_test_prediction = np.mean(test_p, axis=0)
print(f'Mean of all test predictions: {mean_test_prediction}')

0:	learn: 60.0600538	test: 59.0234137	best: 59.0234137 (0)	total: 5.86ms	remaining: 11.7s
100:	learn: 32.5534395	test: 32.6527845	best: 32.6527845 (100)	total: 549ms	remaining: 10.3s
200:	learn: 23.9834687	test: 24.2213988	best: 24.2213988 (200)	total: 1.08s	remaining: 9.68s
300:	learn: 19.8761506	test: 20.2143695	best: 20.2143695 (300)	total: 1.61s	remaining: 9.06s
400:	learn: 17.6973781	test: 18.1569656	best: 18.1569656 (400)	total: 2.12s	remaining: 8.47s
500:	learn: 16.2015358	test: 16.7712718	best: 16.7712718 (500)	total: 2.67s	remaining: 7.99s
600:	learn: 15.0225217	test: 15.7422100	best: 15.7422100 (600)	total: 3.98s	remaining: 9.26s
700:	learn: 14.1387438	test: 15.0040188	best: 15.0040188 (700)	total: 5.3s	remaining: 9.82s
800:	learn: 13.5151733	test: 14.5031954	best: 14.5031954 (800)	total: 6.66s	remaining: 9.96s
900:	learn: 12.9653495	test: 14.0747920	best: 14.0747920 (900)	total: 7.58s	remaining: 9.25s
1000:	learn: 12.4633535	test: 13.6765995	best: 13.6765995 (1000)	total: 8.

In [ ]:
para = {'learning_rate': 0.0670131894596214, 'depth': 8, 'l2_leaf_reg': 0.01240048863005263, 'bagging_temperature': 4.482207609087516, 'border_count': 187, 'random_strength': 0.0006135176867854585, 'od_type': 'IncToDec'}


In [ ]:
maes = []
test_p = []
final_valid_pr ={}

# Train and evaluate the model using KFold
for fold in range(8):
    xtrain=train_new[train_new.kfold!=fold].reset_index(drop=True)
    xvalid=train_new[train_new.kfold==fold].reset_index(drop=True)

    valid_ids=xvalid.candidate_id.values.tolist()

    ytrain=xtrain['triglyceride_lvl']
    yvalid=xvalid['triglyceride_lvl']

    xtrain=xtrain[useful_feature]
    xvalid=xvalid[useful_feature]

    # Initialize and train the CatBoost model
    model = CatBoostRegressor(
        iterations=6000,
        **para,
        eval_metric='MAE',
        random_seed=42,
        verbose=20
    )
    model.fit(xtrain, ytrain,early_stopping_rounds=100 ,eval_set=(xvalid, yvalid), use_best_model=True)

    y_pred = model.predict(xvalid)
    final_valid_pr.update(dict(zip(valid_ids,y_pred)))
    # Calculate MAE
    mae = mean_absolute_error(yvalid,y_pred)
    maes.append(mae)

    # Predict on the new test set for each fold
    test_pred = model.predict(test_new[useful_feature])

    # Store the predictions
    test_p.append(test_pred)

    print(f'Fold MAE: {mae}')

# Calculate mean and std of MAEs
mean_mae = np.mean(maes)
std_mae = np.std(maes)

print(f'Final MAE Mean: {mean_mae}')
print(f'Final MAE Std: {std_mae}')

# Aggregate the predictions to get the mean value for each test sample
mean_test_prediction = np.mean(test_p, axis=0)
print(f'Mean of all test predictions: {mean_test_prediction}')

0:	learn: 58.7300584	test: 57.6593449	best: 57.6593449 (0)	total: 10.6ms	remaining: 1m 3s
20:	learn: 36.2989193	test: 35.9687332	best: 35.9687332 (20)	total: 210ms	remaining: 59.8s
40:	learn: 27.4470195	test: 27.6764754	best: 27.6764754 (40)	total: 431ms	remaining: 1m 2s
60:	learn: 22.4100924	test: 22.8937671	best: 22.8937671 (60)	total: 617ms	remaining: 1m
80:	learn: 19.2524963	test: 19.8222840	best: 19.8222840 (80)	total: 817ms	remaining: 59.7s
100:	learn: 17.4191377	test: 18.1240407	best: 18.1240407 (100)	total: 1s	remaining: 58.6s
120:	learn: 16.1386027	test: 16.9682350	best: 16.9682350 (120)	total: 1.19s	remaining: 57.8s
140:	learn: 15.2533463	test: 16.2320502	best: 16.2320502 (140)	total: 1.44s	remaining: 59.8s
160:	learn: 14.4919874	test: 15.6229494	best: 15.6229494 (160)	total: 2.08s	remaining: 1m 15s
180:	learn: 13.8321096	test: 15.1775354	best: 15.1775354 (180)	total: 3.56s	remaining: 1m 54s
200:	learn: 13.2997108	test: 14.8438715	best: 14.8438715 (200)	total: 4.05s	remaining

In [ ]:
ss['riglyceride_lvl_catnew'] = mean_test_prediction

In [ ]:
ssn=pd.DataFrame()
ssn = ss['candidate_id']
ssn['triglyceride_lvl'] = mean_test_prediction

<ipython-input-102-f51fd8204698>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ssn['triglyceride_lvl'] = mean_test_prediction


In [ ]:
ss

,candidate_id,triglyceride_lvl_xgb,triglyceride_lvl_lgbm,triglyceride_lvl_catboost_2000,riglyceride_lvl_catnew
0,CAN_22401,122.495407,122.473501,120.657863,124.542442
1,CAN_22402,73.831909,78.764994,77.685988,79.104511
2,CAN_22403,63.453911,57.970070,55.311201,51.893239
3,CAN_22404,216.933228,223.288517,214.675940,216.031625
4,CAN_22405,172.554306,190.868276,173.710729,186.543891
...,...,...,...,...,...
9595,CAN_31996,37.773037,34.702801,28.367560,20.707875
9596,CAN_31997,144.394638,144.199243,140.120172,132.775993
9597,CAN_31998,78.454346,75.923327,78.472362,83.358068
9598,CAN_31999,78.415993,81.596139,77.048993,74.534261


In [ ]:
ss['triglyceride_lvl_catboost_2000'] = mean_test_prediction

In [ ]:
ss.to_csv('test_3.csv',index=False)

In [ ]:
final_valid_pred

,candidate_id,triglyceride_lvl_catboost_2000,triglyceride_lvl_xgb,triglyceride_lvl_lgbm
0,CAN_18,57.370589,48.777412,54.555762
1,CAN_35,102.878811,95.671500,97.553547
2,CAN_36,103.518788,91.923600,94.082425
3,CAN_45,46.551205,43.925785,43.477236
4,CAN_55,68.321198,69.359116,75.140224
...,...,...,...,...
22395,CAN_22384,74.484286,72.576130,75.685533
22396,CAN_22390,55.832256,51.681310,40.969546
22397,CAN_22397,214.350290,213.172840,222.469224
22398,CAN_22398,280.650868,255.177630,282.779198


In [ ]:

final_valid_pred=pd.DataFrame.from_dict(final_valid_pr ,orient="index").reset_index()
final_valid_pred.columns=['candidate_id','riglyceride_lvl_catboost_new']
final_valid_pred.to_csv('cat_2.csv',index=False)

In [ ]:
final_valid_pr

{'CAN_211': 102.2784724317193,
 'CAN_314': 242.8332381253198,
 'CAN_323': 114.69636186826067,
 'CAN_352': 340.8059707972909,
 'CAN_424': 238.8618730492829,
 'CAN_452': 64.58603921407865,
 'CAN_829': 208.8017786825935,
 'CAN_1007': 66.78261012481477,
 'CAN_1064': 133.95701990964048,
 'CAN_1098': 131.79722040599341,
 'CAN_1115': 76.69497650220801,
 'CAN_1183': 74.4180181698097,
 'CAN_1511': 60.4991164915768,
 'CAN_1641': 71.56545252691785,
 'CAN_1704': 47.71331189762259,
 'CAN_1711': 190.4915892700998,
 'CAN_1789': 44.06849154636683,
 'CAN_1808': 136.158946073606,
 'CAN_1892': 78.02470945358763,
 'CAN_1902': 87.6262631325007,
 'CAN_1944': 106.59204343241656,
 'CAN_2042': 92.90383070526866,
 'CAN_2146': 71.24327267724438,
 'CAN_2169': 490.2132031647826,
 'CAN_2223': 184.4305456077932,
 'CAN_2239': 235.3140250488434,
 'CAN_2248': 53.56378670419849,
 'CAN_2301': 65.53606811336985,
 'CAN_2326': 172.51576446321724,
 'CAN_2351': 59.38603138951825,
 'CAN_2405': 155.42535999914685,
 'CAN_2409': 

In [ ]:
final_valid_pred

,candidate_id,triglyceride_lvl_catboost_2000
0,CAN_18,57.370589
1,CAN_35,102.878811
2,CAN_36,103.518788
3,CAN_45,46.551205
4,CAN_55,68.321198
...,...,...
22395,CAN_22384,74.484286
22396,CAN_22390,55.832256
22397,CAN_22397,214.350290
22398,CAN_22398,280.650868


In [ ]:
lg = pd.read_csv('/content/lgbm_1.csv')
lg

,candidate_id,triglyceride_lvl_lgbm
0,CAN_18,54.555762
1,CAN_35,97.553547
2,CAN_36,94.082425
3,CAN_45,43.477236
4,CAN_55,75.140224
...,...,...
22395,CAN_22384,75.685533
22396,CAN_22390,40.969546
22397,CAN_22397,222.469224
22398,CAN_22398,282.779198


In [ ]:
xgb_t = pd.read_csv('/content/xgb_tuned_tr.csv')
xgb_t

,candidate_id,triglyceride_lvl_xgb_tuned
0,CAN_18,48.777412
1,CAN_35,95.671500
2,CAN_36,91.923600
3,CAN_45,43.925785
4,CAN_55,69.359116
...,...,...
22395,CAN_22384,72.576130
22396,CAN_22390,51.681310
22397,CAN_22397,213.172840
22398,CAN_22398,255.177630


In [ ]:
ss

,candidate_id,triglyceride_lvl_xgb,triglyceride_lvl_lgbm,triglyceride_lvl_catboost_2000
0,CAN_22401,122.495407,122.473501,120.657863
1,CAN_22402,73.831909,78.764994,77.685988
2,CAN_22403,63.453911,57.970070,55.311201
3,CAN_22404,216.933228,223.288517,214.675940
4,CAN_22405,172.554306,190.868276,173.710729
...,...,...,...,...
9595,CAN_31996,37.773037,34.702801,28.367560
9596,CAN_31997,144.394638,144.199243,140.120172
9597,CAN_31998,78.454346,75.923327,78.472362
9598,CAN_31999,78.415993,81.596139,77.048993


In [ ]:
final_valid_pred['triglyceride_lvl_xgb'] = xgb_t['triglyceride_lvl_xgb_tuned']
final_valid_pred['triglyceride_lvl_lgbm'] = lg['triglyceride_lvl_lgbm']

In [ ]:
final_valid_pred

,candidate_id,triglyceride_lvl_catboost_2000,triglyceride_lvl_xgb,triglyceride_lvl_lgbm
0,CAN_18,57.370589,48.777412,54.555762
1,CAN_35,102.878811,95.671500,97.553547
2,CAN_36,103.518788,91.923600,94.082425
3,CAN_45,46.551205,43.925785,43.477236
4,CAN_55,68.321198,69.359116,75.140224
...,...,...,...,...
22395,CAN_22384,74.484286,72.576130,75.685533
22396,CAN_22390,55.832256,51.681310,40.969546
22397,CAN_22397,214.350290,213.172840,222.469224
22398,CAN_22398,280.650868,255.177630,282.779198


Index(['candidate_id', 'gender', 'age', 'height_in_cm', 'weight_in_lbs',
       'left_eyesight_lvl', 'right_eyesight_lvl', 'can_hear_left_ear',
       'can_hear_right_ear', 'blood_pressure_lvl1', 'blood_pressure_lvl2',
       'glucose_lvl', 'total_cholestrol', 'good_cholestrol_lvl',
       'bad_cholestrol_lvl', 'hemoglobin_lvl', 'urea_lvl', 'creatinine_lvl',
       'liver_enzyme_lvl1', 'liver_enzyme_lvl2', 'smoking_habit',
       'drinking_habit', 'residential_area', 'triglyceride_lvl', 'kfold'],
      dtype='object')

#eng model

In [ ]:
df = train_new.copy()

In [ ]:
aggregated_df = train.groupby('residential_area').agg({
    'triglyceride_lvl': ['mean', 'median', 'max','min','std','count']
})

In [ ]:
aggregated_df

In [ ]:
df = train.copy()
# BMI (Body Mass Index)
df['bmi'] = (df['weight_in_lbs'] / (df['height_in_cm'] / 100) ** 2)

# Mean Eyesight Level
df['mean_eyesight_lvl'] = (df['left_eyesight_lvl'] + df['right_eyesight_lvl']) / 2

# Hearing Ability
df['hearing_ability'] = df['can_hear_left_ear'] + df['can_hear_right_ear']

# Blood Pressure Difference
df['blood_pressure_diff'] = df['blood_pressure_lvl1'] - df['blood_pressure_lvl2']

# Cholesterol Ratios
df['cholesterol_ratio'] = df['total_cholestrol'] / df['good_cholestrol_lvl']
df['bad_good_cholesterol_ratio'] = df['bad_cholestrol_lvl'] / df['good_cholestrol_lvl']

# Liver Enzyme Ratio
df['liver_enzyme_ratio'] = df['liver_enzyme_lvl1'] / df['liver_enzyme_lvl2']

# Habits Encoding
#df['smoking'] = df['smoking_habit'].apply(lambda x: 1 if x == 'yes' else 0)
#df['drinking'] = df['drinking_habit'].apply(lambda x: 1 if x == 'yes' else 0)

# Residential Area Encoding (One-Hot Encoding)
#df = pd.get_dummies(df, columns=['residential_area'], prefix='area')

# Age Group
bins = [0, 30, 50, 100]
labels = ['young', 'middle-aged', 'elderly']
df['age_group'] = pd.cut(df['age'], bins=bins, labels=labels, right=False)



# Now you can use the new DataFrame with engineered features for training your model


In [ ]:
train_new = df.copy()
train_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22400 entries, 0 to 22399
Data columns (total 33 columns):
 #   Column                      Non-Null Count  Dtype   
---  ------                      --------------  -----   
 0   candidate_id                22400 non-null  object  
 1   gender                      22400 non-null  int64   
 2   age                         22400 non-null  int64   
 3   height_in_cm                22400 non-null  int64   
 4   weight_in_lbs               22400 non-null  float64 
 5   left_eyesight_lvl           22400 non-null  float64 
 6   right_eyesight_lvl          22400 non-null  float64 
 7   can_hear_left_ear           22400 non-null  int64   
 8   can_hear_right_ear          22400 non-null  int64   
 9   blood_pressure_lvl1         22400 non-null  float64 
 10  blood_pressure_lvl2         22400 non-null  float64 
 11  glucose_lvl                 22400 non-null  float64 
 12  total_cholestrol            22400 non-null  float64 
 13  good_cholestrol_

In [ ]:
def label_encode_excluding(df, exclude_column):
    # Select columns with object dtype, excluding the specified column
    object_cols = df.select_dtypes(include=['category']).columns
    object_cols = [col for col in object_cols if col != exclude_column]

    # Create a label encoder
    le = LabelEncoder()

    # Label encode the selected object columns
    for col in object_cols:
        df[col] = le.fit_transform(df[col])

    return df

# Label encode object columns, excluding the 'id' column



# Convert object columns to numeric
train_new_n = label_encode_excluding(train_new,'candidate_id')

In [ ]:
train_new_n = label_encode_excluding(train_new,'candidate_id')

In [ ]:
train_new_n.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22400 entries, 0 to 22399
Data columns (total 33 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   candidate_id                22400 non-null  object 
 1   gender                      22400 non-null  int64  
 2   age                         22400 non-null  int64  
 3   height_in_cm                22400 non-null  int64  
 4   weight_in_lbs               22400 non-null  float64
 5   left_eyesight_lvl           22400 non-null  float64
 6   right_eyesight_lvl          22400 non-null  float64
 7   can_hear_left_ear           22400 non-null  int64  
 8   can_hear_right_ear          22400 non-null  int64  
 9   blood_pressure_lvl1         22400 non-null  float64
 10  blood_pressure_lvl2         22400 non-null  float64
 11  glucose_lvl                 22400 non-null  float64
 12  total_cholestrol            22400 non-null  float64
 13  good_cholestrol_lvl         224

In [ ]:
df = test_new.copy()
# BMI (Body Mass Index)
df['bmi'] = (df['weight_in_lbs'] / (df['height_in_cm'] / 100) ** 2)

# Mean Eyesight Level
df['mean_eyesight_lvl'] = (df['left_eyesight_lvl'] + df['right_eyesight_lvl']) / 2

# Hearing Ability
df['hearing_ability'] = df['can_hear_left_ear'] + df['can_hear_right_ear']

# Blood Pressure Difference
df['blood_pressure_diff'] = df['blood_pressure_lvl1'] - df['blood_pressure_lvl2']

# Cholesterol Ratios
df['cholesterol_ratio'] = df['total_cholestrol'] / df['good_cholestrol_lvl']
df['bad_good_cholesterol_ratio'] = df['bad_cholestrol_lvl'] / df['good_cholestrol_lvl']

# Liver Enzyme Ratio
df['liver_enzyme_ratio'] = df['liver_enzyme_lvl1'] / df['liver_enzyme_lvl2']

# Habits Encoding
#df['smoking'] = df['smoking_habit'].apply(lambda x: 1 if x == 'yes' else 0)
#df['drinking'] = df['drinking_habit'].apply(lambda x: 1 if x == 'yes' else 0)

# Residential Area Encoding (One-Hot Encoding)
#df = pd.get_dummies(df, columns=['residential_area'], prefix='area')

# Age Group
bins = [0, 30, 50, 100]
labels = ['young', 'middle-aged', 'elderly']
df['age_group'] = pd.cut(df['age'], bins=bins, labels=labels, right=False)



# Now you can use the new DataFrame with engineered features for training your model


In [ ]:
test_new_n = label_encode_excluding(df,'candidate_id')

In [ ]:

ff=['candidate_id','kfold','triglyceride_lvl','hearing_ability','weight_in_lbs','height_in_cm']
useful_feature=[c for c in train_new_n.columns if c not in ff]
useful_feature

['gender',
 'age',
 'left_eyesight_lvl',
 'right_eyesight_lvl',
 'can_hear_left_ear',
 'can_hear_right_ear',
 'blood_pressure_lvl1',
 'blood_pressure_lvl2',
 'glucose_lvl',
 'total_cholestrol',
 'good_cholestrol_lvl',
 'bad_cholestrol_lvl',
 'hemoglobin_lvl',
 'urea_lvl',
 'creatinine_lvl',
 'liver_enzyme_lvl1',
 'liver_enzyme_lvl2',
 'smoking_habit',
 'drinking_habit',
 'residential_area',
 'bmi',
 'mean_eyesight_lvl',
 'blood_pressure_diff',
 'cholesterol_ratio',
 'bad_good_cholesterol_ratio',
 'liver_enzyme_ratio',
 'age_group']

In [ ]:
maes = []
test_p = []
final_valid_pr ={}
train_new = train_new_n.copy()
test_new = df.copy()
# Train and evaluate the model using KFold
maes = []
test_p = []
final_valid_pr ={}

# Train and evaluate the model using KFold
for fold in range(8):
    xtrain=train_new[train_new.kfold!=fold].reset_index(drop=True)
    xvalid=train_new[train_new.kfold==fold].reset_index(drop=True)

    valid_ids=xvalid.candidate_id.values.tolist()

    ytrain=xtrain['triglyceride_lvl']
    yvalid=xvalid['triglyceride_lvl']

    xtrain=xtrain[useful_feature]
    xvalid=xvalid[useful_feature]

    # Initialize and train the CatBoost model
    model = CatBoostRegressor(
        iterations=6000,
        **para,
        eval_metric='MAE',
        random_seed=42,
        verbose=20
    )
    model.fit(xtrain, ytrain,early_stopping_rounds=100 ,eval_set=(xvalid, yvalid), use_best_model=True)

    y_pred = model.predict(xvalid)
    final_valid_pr.update(dict(zip(valid_ids,y_pred)))
    # Calculate MAE
    mae = mean_absolute_error(yvalid,y_pred)
    maes.append(mae)

    # Predict on the new test set for each fold
    test_pred = model.predict(test_new[useful_feature])

    # Store the predictions
    test_p.append(test_pred)

    print(f'Fold MAE: {mae}')

# Calculate mean and std of MAEs
mean_mae = np.mean(maes)
std_mae = np.std(maes)

print(f'Final MAE Mean: {mean_mae}')
print(f'Final MAE Std: {std_mae}')

# Aggregate the predictions to get the mean value for each test sample
mean_test_prediction = np.mean(test_p, axis=0)
print(f'Mean of all test predictions: {mean_test_prediction}')

0:	learn: 58.4100271	test: 57.4127214	best: 57.4127214 (0)	total: 13ms	remaining: 1m 17s
20:	learn: 34.0142830	test: 33.6742306	best: 33.6742306 (20)	total: 267ms	remaining: 1m 15s
40:	learn: 25.1429013	test: 25.2754605	best: 25.2754605 (40)	total: 510ms	remaining: 1m 14s
60:	learn: 20.4990471	test: 20.8957728	best: 20.8957728 (60)	total: 758ms	remaining: 1m 13s
80:	learn: 17.9292384	test: 18.5643769	best: 18.5643769 (80)	total: 1.01s	remaining: 1m 14s
100:	learn: 15.9934705	test: 16.8537887	best: 16.8537887 (100)	total: 1.26s	remaining: 1m 13s
120:	learn: 14.7105688	test: 15.8063124	best: 15.8063124 (120)	total: 1.51s	remaining: 1m 13s
140:	learn: 13.7365811	test: 14.9997382	best: 14.9997382 (140)	total: 1.74s	remaining: 1m 12s
160:	learn: 12.9602187	test: 14.4305461	best: 14.4305461 (160)	total: 2.08s	remaining: 1m 15s
180:	learn: 12.3122988	test: 13.9272924	best: 13.9272924 (180)	total: 2.73s	remaining: 1m 27s
200:	learn: 11.7972857	test: 13.5630399	best: 13.5563615 (199)	total: 3.4

In [ ]:
test_new.columns

Index(['candidate_id', 'gender', 'age', 'height_in_cm', 'weight_in_lbs',
       'left_eyesight_lvl', 'right_eyesight_lvl', 'can_hear_left_ear',
       'can_hear_right_ear', 'blood_pressure_lvl1', 'blood_pressure_lvl2',
       'glucose_lvl', 'total_cholestrol', 'good_cholestrol_lvl',
       'bad_cholestrol_lvl', 'hemoglobin_lvl', 'urea_lvl', 'creatinine_lvl',
       'liver_enzyme_lvl1', 'liver_enzyme_lvl2', 'smoking_habit',
       'drinking_habit', 'residential_area', 'bmi', 'mean_eyesight_lvl',
       'hearing_ability', 'blood_pressure_diff', 'cholesterol_ratio',
       'bad_good_cholesterol_ratio', 'liver_enzyme_ratio', 'age_group'],
      dtype='object')

In [ ]:
ss

,candidate_id,triglyceride_lvl
0,CAN_22401,116
1,CAN_22402,78
2,CAN_22403,42
3,CAN_22404,222
4,CAN_22405,194


In [ ]:
ssd

0                                                           CAN_22401
1                                                           CAN_22402
2                                                           CAN_22403
3                                                           CAN_22404
4                                                           CAN_22405
                                          ...                        
9596                                                        CAN_31997
9597                                                        CAN_31998
9598                                                        CAN_31999
9599                                                        CAN_32000
triglyceride_lvl    [124.5389364587385, 78.84977994645497, 46.2297...
Name: candidate_id, Length: 9601, dtype: object

In [ ]:
mean_test_prediction


array([124.53893646,  78.84977995,  46.22975852, ...,  78.97774736,
        70.60982813, 163.41139321])

In [ ]:
ssd = pd.DataFrame()
ssd['candidate_id'] = test_new.candidate_id
ssd['triglyceride_lvl'] = mean_test_prediction
ssd
ssd.to_csv('cat_fea_new.csv',index=False)

In [ ]:
ssd

0                                                           CAN_22401
1                                                           CAN_22402
2                                                           CAN_22403
3                                                           CAN_22404
4                                                           CAN_22405
                                          ...                        
9596                                                        CAN_31997
9597                                                        CAN_31998
9598                                                        CAN_31999
9599                                                        CAN_32000
triglyceride_lvl    [124.5389364587385, 78.84977994645497, 46.2297...
Name: candidate_id, Length: 9601, dtype: object

# stacing the model predictons

In [ ]:
df_tr = final_valid_pred.merge(train[['candidate_id','triglyceride_lvl']], on='candidate_id', how='inner')
df_tr['kfold'] = train_new['kfold']

In [ ]:
df_tr

,candidate_id,triglyceride_lvl_catboost_2000,triglyceride_lvl_xgb,triglyceride_lvl_lgbm,triglyceride_lvl,kfold
0,CAN_18,57.370589,48.777412,54.555762,54.0,2
1,CAN_35,102.878811,95.671500,97.553547,86.0,5
2,CAN_36,103.518788,91.923600,94.082425,110.0,4
3,CAN_45,46.551205,43.925785,43.477236,50.0,1
4,CAN_55,68.321198,69.359116,75.140224,65.0,2
...,...,...,...,...,...,...
22395,CAN_22384,74.484286,72.576130,75.685533,72.0,1
22396,CAN_22390,55.832256,51.681310,40.969546,53.0,7
22397,CAN_22397,214.350290,213.172840,222.469224,195.0,7
22398,CAN_22398,280.650868,255.177630,282.779198,328.0,1


In [ ]:
df_tr

,candidate_id,triglyceride_lvl_catboost_2000,triglyceride_lvl_xgb,triglyceride_lvl_lgbm,triglyceride_lvl,kfold,riglyceride_lvl_catnew
0,CAN_18,57.370589,48.777412,54.555762,54.0,2,54.880748
1,CAN_35,102.878811,95.671500,97.553547,86.0,5,88.420873
2,CAN_36,103.518788,91.923600,94.082425,110.0,4,94.308846
3,CAN_45,46.551205,43.925785,43.477236,50.0,1,44.441564
4,CAN_55,68.321198,69.359116,75.140224,65.0,2,66.349909
...,...,...,...,...,...,...,...
22395,CAN_22384,74.484286,72.576130,75.685533,72.0,1,74.713517
22396,CAN_22390,55.832256,51.681310,40.969546,53.0,7,46.286614
22397,CAN_22397,214.350290,213.172840,222.469224,195.0,7,217.840542
22398,CAN_22398,280.650868,255.177630,282.779198,328.0,1,283.478294


In [ ]:
values = list(final_valid_pr.values())
values

[53.106486494928774,
 99.44623895694605,
 102.68658626338392,
 46.30930879390232,
 62.72104125885022,
 72.64896523956372,
 165.2150766690751,
 215.32372400304496,
 48.14848191361763,
 153.84122656967241,
 169.6128033595742,
 149.65625801412605,
 87.11145992635332,
 84.75720473348201,
 168.78490591755278,
 241.8743795859969,
 190.09281845555344,
 99.33352357066752,
 77.64137876345089,
 119.37908202199537,
 50.11271836239894,
 55.447132310634856,
 99.89114050558177,
 80.20433133787319,
 375.18653894767203,
 107.12328219805273,
 288.9488366585788,
 85.05621737985727,
 185.28905976311611,
 332.20049351221456,
 118.97431189011077,
 72.47925784810687,
 103.69257461631612,
 43.26442557844945,
 288.9804374547676,
 91.18874527372043,
 145.87542671133477,
 71.05849239481796,
 102.84756775162265,
 126.84361138012616,
 169.38655139365213,
 75.56570617473048,
 88.46622980333112,
 272.68107839678885,
 327.3577542833512,
 93.61776711543992,
 64.04688768875653,
 100.4133727395624,
 66.75271670858588,


In [ ]:
df_tr['triglyceride_lvl_catfea'] = values

In [ ]:
df_tr.to_csv('5model_stack_tr_nn.csv',index=False)

In [ ]:
df_tr

,candidate_id,triglyceride_lvl_catboost_2000,triglyceride_lvl_xgb,triglyceride_lvl_lgbm,triglyceride_lvl,kfold,riglyceride_lvl_catnew
0,CAN_18,57.370589,48.777412,54.555762,54.0,2,54.880748
1,CAN_35,102.878811,95.671500,97.553547,86.0,5,88.420873
2,CAN_36,103.518788,91.923600,94.082425,110.0,4,94.308846
3,CAN_45,46.551205,43.925785,43.477236,50.0,1,44.441564
4,CAN_55,68.321198,69.359116,75.140224,65.0,2,66.349909
...,...,...,...,...,...,...,...
22395,CAN_22384,74.484286,72.576130,75.685533,72.0,1,74.713517
22396,CAN_22390,55.832256,51.681310,40.969546,53.0,7,46.286614
22397,CAN_22397,214.350290,213.172840,222.469224,195.0,7,217.840542
22398,CAN_22398,280.650868,255.177630,282.779198,328.0,1,283.478294


In [ ]:
df_tr.columns

Index(['candidate_id', 'triglyceride_lvl_catboost_2000',
       'triglyceride_lvl_xgb', 'triglyceride_lvl_lgbm', 'triglyceride_lvl',
       'kfold', 'riglyceride_lvl_catnew'],
      dtype='object')

In [ ]:
ss

,candidate_id,triglyceride_lvl_xgb,triglyceride_lvl_lgbm,triglyceride_lvl_catboost_2000,riglyceride_lvl_catnew
0,CAN_22401,122.495407,122.473501,120.657863,123.154213
1,CAN_22402,73.831909,78.764994,77.685988,72.782346
2,CAN_22403,63.453911,57.970070,55.311201,52.299489
3,CAN_22404,216.933228,223.288517,214.675940,225.626019
4,CAN_22405,172.554306,190.868276,173.710729,178.285346
...,...,...,...,...,...
9595,CAN_31996,37.773037,34.702801,28.367560,46.168210
9596,CAN_31997,144.394638,144.199243,140.120172,143.767568
9597,CAN_31998,78.454346,75.923327,78.472362,75.084476
9598,CAN_31999,78.415993,81.596139,77.048993,73.116309


In [ ]:
ss_new = ss[['triglyceride_lvl_catboost_2000','triglyceride_lvl_xgb', 'triglyceride_lvl_lgbm','riglyceride_lvl_catnew']]

In [ ]:
dd = pd.read_csv('/content/test_3.csv')

In [ ]:
rtt = pd.read_csv('/content/stack_4.csv')
rtt

,candidate_id,triglyceride_lvl
0,CAN_22401,122.042838
1,CAN_22402,76.787612
2,CAN_22403,54.355448
3,CAN_22404,216.783053
4,CAN_22405,178.579867
...,...,...
9595,CAN_31996,25.908206
9596,CAN_31997,138.561990
9597,CAN_31998,79.216305
9598,CAN_31999,75.715771


In [ ]:
dd['riglyceride_lvl_catnew'] = mean_test_prediction

In [ ]:
ssd

,candidate_id,triglyceride_lvl
0,CAN_22401,124.538936
1,CAN_22402,78.849780
2,CAN_22403,46.229759
3,CAN_22404,213.015492
4,CAN_22405,207.656383
...,...,...
9595,CAN_31996,46.071456
9596,CAN_31997,131.626503
9597,CAN_31998,78.977747
9598,CAN_31999,70.609828


In [ ]:
dd['triglyceride_lvl_catfea'] = ssd['triglyceride_lvl']

In [ ]:
dd

,candidate_id,triglyceride_lvl_xgb,triglyceride_lvl_lgbm,triglyceride_lvl_catboost_2000
0,CAN_22401,122.495410,122.473501,120.657863
1,CAN_22402,73.831910,78.764994,77.685988
2,CAN_22403,63.453910,57.970070,55.311201
3,CAN_22404,216.933230,223.288517,214.675940
4,CAN_22405,172.554300,190.868276,173.710729
...,...,...,...,...
9595,CAN_31996,37.773037,34.702801,28.367560
9596,CAN_31997,144.394640,144.199243,140.120172
9597,CAN_31998,78.454346,75.923327,78.472362
9598,CAN_31999,78.415990,81.596139,77.048993


In [ ]:
ss_new

,candidate_id,triglyceride_lvl
0,CAN_22401,122.042838
1,CAN_22402,76.787612
2,CAN_22403,54.355448
3,CAN_22404,216.783053
4,CAN_22405,178.579867
...,...,...
9595,CAN_31996,25.908206
9596,CAN_31997,138.561990
9597,CAN_31998,79.216305
9598,CAN_31999,75.715771


In [ ]:
ff=['candidate_id','kfold','triglyceride_lvl']
useful_feature=[c for c in df_tr.columns if c not in ff]
useful_feature

['triglyceride_lvl_catboost_2000',
 'triglyceride_lvl_xgb',
 'triglyceride_lvl_lgbm',
 'riglyceride_lvl_catnew',
 'triglyceride_lvl_catfea']

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
maes = []
test_p = []
final_valid_pr ={}
train_new = df_tr.copy()
test_new = dd.copy()
# Train and evaluate the model using KFold
for fold in range(8):
    xtrain=train_new[train_new.kfold!=fold].reset_index(drop=True)
    xvalid=train_new[train_new.kfold==fold].reset_index(drop=True)

    #valid_ids=xvalid.candidate_id.values.tolist()

    ytrain=xtrain['triglyceride_lvl']
    yvalid=xvalid['triglyceride_lvl']

    xtrain=xtrain[useful_feature]
    xvalid=xvalid[useful_feature]

    # Initialize and train the CatBoost model
    model = LinearRegression()
    model.fit(xtrain, ytrain)

    y_pred = model.predict(xvalid)
    #final_valid_pr.update(dict(zip(valid_ids,y_pred)))
    # Calculate MAE
    mae = mean_absolute_error(yvalid,y_pred)
    maes.append(mae)

    # Predict on the new test set for each fold
    test_pred = model.predict(test_new[useful_feature])

    # Store the predictions
    test_p.append(test_pred)

    print(f'Fold MAE: {mae}')

# Calculate mean and std of MAEs
mean_mae = np.mean(maes)
std_mae = np.std(maes)

print(f'Final MAE Mean: {mean_mae}')
print(f'Final MAE Std: {std_mae}')

# Aggregate the predictions to get the mean value for each test sample
mean_test_prediction = np.mean(test_p, axis=0)
print(f'Mean of all test predictions: {mean_test_prediction}')

Fold MAE: 10.899044290272405
Fold MAE: 10.80199023372053
Fold MAE: 10.538819479267742
Fold MAE: 10.830954367182125
Fold MAE: 10.600022532532623
Fold MAE: 10.546935228095862
Fold MAE: 10.01152145101077
Fold MAE: 10.485340902335766
Final MAE Mean: 10.589328560552229
Final MAE Std: 0.2618847701671638
Mean of all test predictions: [122.59758131  77.20056861  51.46765448 ...  78.82394233  73.85166628
 165.18069426]


In [ ]:
ss_new = dd[['candidate_id']]
ss_new['triglyceride_lvl'] = mean_test_prediction
ss_new.to_csv('stack_5_correct.csv',index=False)

<ipython-input-172-4db36be14c8b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ss_new['triglyceride_lvl'] = mean_test_prediction


#ex

In [ ]:
df=train_new.copy()
scores=[]
test_r=test_new[useful_feature]
final_test_pr=[]
final_valid_pr={}

for fold in range(8):
  xtrain=df[df.kfold!=fold].reset_index(drop=True)
  xvalid=df[df.kfold==fold].reset_index(drop=True)
  #
  valid_ids=xvalid.id.values.tolist()

  ytrain=xtrain['Target']
  yvalid=xvalid['Target']

  xtrain=xtrain[useful_feature]
  xvalid=xvalid[useful_feature]



  lgbm = LGBMRegressor(**param_new)

  model=lgbm.fit(xtrain, ytrain ,eval_set=[(xvalid, yvalid)], verbose=20, early_stopping_rounds=200)
  #model=lgbm.fit(xtrain, ytrain)

  valid_pred = model.predict(xvalid)

  rmse=np.sqrt(mean_squared_error(yvalid,valid_pred))
  print(fold,rmse)
  #print("RMSLE LGBM: ", sqrt((mean_squared_error((yvalid),(valid_pred)))))

  test_p=model.predict(test_r)
  final_test_pr.append(test_p)

  final_valid_pr.update(dict(zip(valid_ids,valid_pred)))

  scores.append(rmse)

print(np.mean(scores),np.std(scores))

final_valid_pred=pd.DataFrame.from_dict(final_valid_pr ,orient="index").reset_index()
final_valid_pred.columns=['id','pred_4']
final_valid_pred.to_csv('valid_pr_4.csv',index=False)

sub_new['pred_4']=np.mean(np.column_stack(final_test_pr),axis=1)